In [ ]:
import torch
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely.ops import unary_union
from scipy.spatial import Voronoi
from pyproj import Transformer
import json
from shapely.strtree import STRtree
from collections import Counter
from sklearn.cluster import DBSCAN
import pandas as pd
import geopandas as gpd

In [ ]:
def load_poi_points(poi_json_file):
    with open(poi_json_file, 'r') as f:
        poi_data = json.load(f)
    
    points = [Point(feature['geometry']['coordinates']) for feature in poi_data['features']]
    return points

def categorize_pois(poi_json_file, top_k=5):
    with open(poi_json_file, 'r') as f:
        poi_data = json.load(f)
    
    categories = {}
    for feature in poi_data['features']:
        try:
            category = feature['properties']['categories']['primary']
            point = Point(feature['geometry']['coordinates'])
            if category not in categories:
                categories[category] = []
            categories[category].append(point)
        except:
            pass

    # Get top 5 categories
    category_counts = {cat: len(pts) for cat, pts in categories.items()}
    top_k_categories = [cat for cat, count in Counter(category_counts).most_common(top_k)]
    
    filtered_categories = {cat: pts for cat, pts in categories.items() if cat in top_k_categories}

    return filtered_categories

def build_convex_hull_boundary_from_poi(points):
    points_gdf = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")
    polygon_boundary = points_gdf.unary_union.convex_hull
    return polygon_boundary

def buffer_polygon_boundary(polygon_boundary, buffer_distance=50):
    buffered_polygon = polygon_boundary.buffer(buffer_distance)
    return buffered_polygon

def create_voronoi_torch(points, boundary, device="cpu"):
    if len(points) < 3:
        return None
    coords = torch.tensor([[p.x, p.y] for p in points], device=device)
    coords_np = coords.cpu().numpy()
    vor = Voronoi(coords_np)
    
    polygons = []
    for region_idx in vor.regions:
        if not region_idx or -1 in region_idx:
            continue
        polygon_points = [vor.vertices[i] for i in region_idx]
        poly = Polygon(polygon_points)
        clipped_poly = poly.intersection(boundary)
        if not clipped_poly.is_empty:
            polygons.append(clipped_poly)
    return polygons

def generate_points_from_voronoi_torch(voronoi_polygons_list, device="cpu", distance_threshold=1e-15):
    points = []
    
    valid_polygons = []
    for poly in voronoi_polygons_list:
        if isinstance(poly, (Polygon, MultiPolygon)):
            if not poly.is_empty:
                valid_polygons.append(poly)
    
    rtree = STRtree(valid_polygons)
    
    # Process each valid polygon
    for poly in valid_polygons:
        if isinstance(poly, Polygon):
            centroid = torch.tensor([poly.centroid.x, poly.centroid.y], device=device)
            distance_to_boundary = poly.boundary.distance(poly.centroid)
            
            if distance_to_boundary < float(distance_threshold):
                # Intersection points: High-density regions
                try:
                    nearby_polygons = rtree.query(poly)
                    for poly2 in nearby_polygons:
                        if poly != poly2:
                            intersection = poly.intersection(poly2)
                            if isinstance(intersection, Point):
                                points.append([intersection.x, intersection.y])
                            elif isinstance(intersection, MultiPolygon):
                                points.extend([[p.x, p.y] for p in intersection])
                except:
                    pass
            else:
                # Far-from-boundary: Centroid
                points.append(centroid.cpu().numpy())
    return points

def merge_points(points_list):
    valid_points = []
    for p in points_list:
        valid_points.append(Point(p[0], p[1]))
    merged_geometry = unary_union(valid_points)
    return merged_geometry

def validate_points(points, poi_buffer, boundary):
    validated_points = []

    if isinstance(points, Point):
        if poi_buffer.contains(points) and boundary.contains(points):
            validated_points.append(points)
    elif isinstance(points, MultiPoint):
        for point in points.geoms:
            if poi_buffer.contains(point) and boundary.contains(point):
                validated_points.append(point)
    return validated_points

def create_output_schema(output_file, validated_points, transformer, eps1=0.01, eps2=0.01, eps3=0.03, min_samples1=3, min_samples2=4, min_samples3=6):
    coordinates = [(poi.y, poi.x) for poi in validated_points]

    # Apply DBSCAN
    dbscan1 = DBSCAN(eps=eps1, min_samples=min_samples1)
    labels1 = dbscan1.fit_predict(coordinates)
    
    # Prepare 2 lists for distinct types
    first_list_unhandled = []
    second_stage_clusters = []

    # Prepare 2 lists for preliminary outputs between clustering
    first_list_for_clustering = []
    second_list_for_clustering = []
    output_counter = 0

    for idx, label1 in enumerate(labels1):
        if label1 == -1:
            first_list_for_clustering.append((idx, coordinates[idx], label1))
        else:
            second_list_for_clustering.append((idx, coordinates[idx], label1))
    
    first_stage_clusters = {}
    for idx, point, label1 in first_list_for_clustering:
        if label1 not in first_stage_clusters:
            first_stage_clusters[label1] = []
        first_stage_clusters[label1].append((idx, point))
    
    for label1, cluster_points in first_stage_clusters.items():
        points_for_second_dbscan = [p[1] for p in cluster_points]
        
        if len(points_for_second_dbscan) > 1:
            # Apply second DBSCAN
            dbscan3 = DBSCAN(eps=eps3, min_samples=min_samples3)
            labels3 = dbscan3.fit_predict(points_for_second_dbscan)
            
            unique_labels3 = set(labels3)
            for label3 in unique_labels3:
                cluster_subpoints = [points_for_second_dbscan[i] for i, lbl in enumerate(labels3) if lbl == label3]
                
                # Calculate centroid of the second-stage clusters
                centroid_lat = sum([p[0] for p in cluster_subpoints]) / len(cluster_subpoints)
                centroid_lon = sum([p[1] for p in cluster_subpoints]) / len(cluster_subpoints)
                
                x, y = transformer.transform(centroid_lat, centroid_lon)
                
                # Assign capacity based on whether it's merged or not
                capacity = 2 if len(cluster_subpoints) > 1 else 0
                
                if capacity > 0:
                    row = {
                        'id': str(output_counter),
                        'latitude': round(centroid_lat, 6),
                        'longitude': round(centroid_lon, 6),
                        'capacity': capacity,
                        'geometry': Point(x, y)
                        #'geometry': Point(centroid_lon, centroid_lat)
                    }
                    first_list_unhandled.append(row)
                    output_counter += 1


    first_stage_clusters = {}
    for idx, point, label1 in second_list_for_clustering:
        if label1 not in first_stage_clusters:
            first_stage_clusters[label1] = []
        first_stage_clusters[label1].append((idx, point))

    for label1, cluster_points in first_stage_clusters.items():
        points_for_second_dbscan = [p[1] for p in cluster_points]
        
        if len(points_for_second_dbscan) > 1:
            # Apply second DBSCAN for tighter clustering
            dbscan2 = DBSCAN(eps=eps2, min_samples=min_samples2)
            labels2 = dbscan2.fit_predict(points_for_second_dbscan)
            
            unique_labels2 = set(labels2)
            for label2 in unique_labels2:
                cluster_subpoints = [points_for_second_dbscan[i] for i, lbl in enumerate(labels2) if lbl == label2]
                
                # Calculate centroid of the second-stage clusters
                centroid_lat = sum([p[0] for p in cluster_subpoints]) / len(cluster_subpoints)
                centroid_lon = sum([p[1] for p in cluster_subpoints]) / len(cluster_subpoints)
                
                x, y = transformer.transform(centroid_lat, centroid_lon)
                
                # Assign capacity based on whether it's merged or not
                capacity = 5 if len(cluster_subpoints) > 1 else 4
                
                row = {
                    'id': str(output_counter),
                    'latitude': round(centroid_lat, 6),
                    'longitude': round(centroid_lon, 6),
                    'capacity': capacity,
                    'geometry': Point(x, y)
                    #'geometry': Point(centroid_lon, centroid_lat)
                }
                second_stage_clusters.append(row)
                output_counter += 1
        else:
            # If the cluster has only one point, treat it as unmerged
            lat, lon = points_for_second_dbscan[0]
            x, y = transformer.transform(lat, lon)
            row = {
                'id': str(output_counter),
                'latitude': round(lat, 6),
                'longitude': round(lon, 6),
                'capacity': 3,
                'geometry': Point(x, y)
                #'geometry': Point(lon, lat)
            }
            first_list_unhandled.append(row)
            output_counter += 1

    # Combine first and second stage outputs
    final_output = first_list_unhandled + second_stage_clusters

    df = gpd.GeoDataFrame(final_output)
    df = df[['id', 'latitude', 'longitude', 'capacity', 'geometry']]

    df.to_file(output_file, driver="GPKG", index=False)

    return True



def main(poi_json_file, output_geopkg_file, top_k=5, buffer_distance=5, distance_threshold=1e-15, device="cpu"):
    # Load POI data
    points = load_poi_points(poi_json_file)
    polygon_boundary = build_convex_hull_boundary_from_poi(points)
    buffered_polygon = buffer_polygon_boundary(polygon_boundary, buffer_distance)

    # Categorize POIs by category
    categories = categorize_pois(poi_json_file, top_k)

    # For each category, generate a Voronoi diagram
    merged_points = []
    for category, category_points in categories.items():
        voronoi_polygons = create_voronoi_torch(category_points, buffered_polygon, device=device)
        voronoi_points = generate_points_from_voronoi_torch(voronoi_polygons, device=device, distance_threshold=distance_threshold)
        merged_points.extend(voronoi_points)

    # Merge and validate
    merged_points = merge_points(merged_points)
    poi_buffer = unary_union([p.buffer(buffer_distance) for p in points])
    validated_points = validate_points(merged_points, poi_buffer, polygon_boundary)
    
    # Hierachical cluster and generate the output 
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    create_output_schema(output_geopkg_file, validated_points, transformer)

In [ ]:
poi_json_file = "ge-package-tojson-out.json"
output_geopkg_file = "ev_charging_stations_isi_i2.gpkg"
    
main(poi_json_file, output_geopkg_file, device="cuda" if torch.cuda.is_available() else "cpu")